In [1]:
import os
# 用 os.chdir() 跳到 main
os.chdir('../main')

In [2]:
model_name = "PPO"
checkpoints = 2500000
model_checkpoints = model_name + "_" + str(checkpoints) + "_steps" # Speicify the model file to load. Model "ppo_ryu_2500000_steps_updated" is capable of beating the final stage (Bison) of the game.
# Model notes:
# ppo_ryu_2000000_steps_updated: Just beginning to overfit state, generalizable but not quite capable.
# ppo_ryu_2500000_steps_updated: Approaching the final overfitted state, cannot dominate first round but partially generalizable. High chance of beating the final stage.
# ppo_ryu_3000000_steps_updated: Near the final overfitted state, almost dominate first round but barely generalizable.
# ppo_ryu_7000000_steps_updated: Overfitted, dominates first round but not generalizable. 


RENDERING = False    # Whether to render the game screen.
RESET_ROUND = True  # Whether to reset the round when fight is over. 

RANDOM_ACTION = False
NUM_EPISODES = 100 # Make sure NUM_EPISODES >= 3 if you set RESET_ROUND to False to see the whole final stage game.
MODEL_DIR = "trained_models_" + model_name +"/"
# MODEL_DIR = r"trained_models/"


In [3]:
from pyvirtualdisplay import Display

if RENDERING:
    virtual_display = Display(visible=0, size=(1400, 900))
    virtual_display.start()

%matplotlib inline
import matplotlib.pyplot as plt

from IPython import display

In [4]:
# Copyright 2023 LIN Yi. All Rights Reserved.
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

import os
import time 

import retro
from stable_baselines3 import PPO

from street_fighter_custom_wrapper import StreetFighterCustomWrapper

def make_env(game, state):
    def _init():
        env = retro.make(
            game=game, 
            state=state, 
            use_restricted_actions=retro.Actions.FILTERED,
            obs_type=retro.Observations.IMAGE
        )
        env = StreetFighterCustomWrapper(env, reset_round=RESET_ROUND, rendering=RENDERING)
        return env
    return _init

game = "StreetFighterIISpecialChampionEdition-Genesis"
env = make_env(game, state="Champion.Level12.RyuVsBison")()
# model = PPO("CnnPolicy", env)

if not RANDOM_ACTION:
    model = PPO.load(os.path.join(MODEL_DIR, model_checkpoints), env=env)

obs = env.reset()
done = False

num_episodes = NUM_EPISODES
episode_reward_sum = 0
num_victory = 0

print("\nFighting Begins!\n")

for _ in range(num_episodes):
    done = False
    
    if RESET_ROUND:
        obs = env.reset()

    total_reward = 0

    if RENDERING:
        img = plt.imshow(env.render(mode='rgb_array'))

    while not done:
        timestamp = time.time()

        if RANDOM_ACTION:
            # sample action from action space
            # obs, reward, done, info = env.step(env.action_space.sample())

            # test specified action
            # button_combos [[0, 16, 32], [0, 64, 128], [0, 1, 2, 3, 256, 257, 512, 513, 1024, 1026, 1536, 2048, 2304, 2560]]
            # buttons ['B', 'A', 'MODE', 'START', 'UP', 'DOWN', 'LEFT', 'RIGHT', 'C', 'Y', 'X', 'Z']
            test_action = [0,0,0,0,1,0,0,0,0,0,0,0]
            obs, reward, done, info = env.step(test_action)
        else:
            action, _states = model.predict(obs)
            # print(action)
            obs, reward, done, info = env.step(action)

        if reward != 0:
            total_reward += reward
            # print("Reward: {:.3f}, playerHP: {}, enemyHP:{}".format(reward, info['agent_hp'], info['enemy_hp']))
        
        # done should be returned by env.step() when the game is over
        # if info['enemy_hp'] < 0 or info['agent_hp'] < 0:
        #     done = True

        if RENDERING:
            img.set_data(env.render(mode='rgb_array'))
            display.display(plt.gcf())
            display.clear_output(wait=True)

    if info['enemy_hp'] < 0:
        print(_, "Victory!")
        num_victory += 1
    else:
        print(_, "Lose...")

    # print("Total reward: {}\n".format(total_reward))
    episode_reward_sum += total_reward

    if not RESET_ROUND:
        while info['enemy_hp'] < 0 or info['agent_hp'] < 0:
        # Inter scene transition. Do nothing.
            obs, reward, done, info = env.step([0] * 12)
            if RENDERING:
                env.render(mode='rgb_array')

env.close()

print("\nFighting Ends!\n")
print(model_checkpoints)
print("Winning rate: {}".format(1.0 * num_victory / num_episodes))
if RANDOM_ACTION:
    print("Average reward for random action: {}".format(episode_reward_sum/num_episodes))
else:
    print("Average reward for {}: {}".format(model_checkpoints, episode_reward_sum/num_episodes))

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.

Fighting Begins!

0 Victory!
1 Victory!
2 Lose...
3 Victory!
4 Victory!
5 Victory!
6 Victory!
7 Victory!
8 Victory!
9 Victory!
10 Victory!
11 Lose...
12 Lose...
13 Victory!
14 Lose...
15 Lose...
16 Victory!
17 Lose...
18 Victory!
19 Victory!
20 Victory!
21 Lose...
22 Lose...
23 Victory!
24 Victory!
25 Victory!
26 Lose...
27 Victory!
28 Victory!
29 Victory!
30 Lose...
31 Victory!
32 Lose...
33 Victory!
34 Victory!
35 Victory!
36 Victory!
37 Lose...
38 Victory!
39 Lose...
40 Victory!
41 Victory!
42 Victory!
43 Victory!
44 Victory!
45 Victory!
46 Victory!
47 Victory!
48 Victory!
49 Victory!
50 Victory!
51 Victory!
52 Victory!
53 Victory!
54 Victory!
55 Lose...
56 Victory!
57 Victory!
58 Victory!
59 Victory!
60 Victory!
61 Victory!
62 Victory!
63 Victory!
64 Victory!
65 Victory!
66 Victory!
67 Victory!
68 Victory!
69 Victory!
70 Lose...
71 Victory!
72 Victory!
73 Victory!


In [5]:
# if interrupt by KeyboardInterrupt, close the env
env.close()